## AI Agents

In [1]:
!pip install openai -q

In [2]:
import os
from google.colab import userdata
from openai import OpenAI
import yfinance as yf

# Load API key from Colab Secrets
os.environ["OPENAI_API_KEY"] = userdata.get("openai_api_key")

client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

MODEL_NAME = "gpt-4o-mini"


In [3]:
def call_llm(prompt, temperature=0.4):
    """Simple LLM wrapper."""
    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[{"role": "user", "content": prompt}],
        temperature=temperature
    )
    return response.choices[0].message.content

In [4]:
#  HELPER: FORMAT QUANT DATA FOR PROMPTS

def format_quantitative_for_prompt(q):
    if not q:
        return "No reliable numeric data was retrieved."

    lines = []

    if q.get("current_price") is not None:
        lines.append(f"- Current share price: {q['current_price']:.2f} USD")
    if q.get("market_cap") is not None:
        lines.append(f"- Market cap: {q['market_cap'] / 1e9:.1f} Bn USD")
    if q.get("pe_ratio") is not None:
        lines.append(f"- Trailing P/E ratio: {q['pe_ratio']:.1f}x")
    if q.get("eps_trailing") is not None:
        lines.append(f"- Trailing EPS: {q['eps_trailing']:.2f} USD")
    if q.get("revenue_ttm") is not None:
        lines.append(f"- TTM revenue: {q['revenue_ttm'] / 1e9:.1f} Bn USD")
    if q.get("gross_margins") is not None:
        lines.append(f"- Gross margin: {q['gross_margins'] * 100:.1f}%")
    if q.get("operating_margins") is not None:
        lines.append(f"- Operating margin: {q['operating_margins'] * 100:.1f}%")
    if q.get("free_cash_flow") is not None:
        lines.append(f"- Free cash flow: {q['free_cash_flow'] / 1e9:.2f} Bn USD")
    if q.get("52wk_high") is not None:
        lines.append(f"- 52-week high: {q['52wk_high']:.2f} USD")
    if q.get("52wk_low") is not None:
        lines.append(f"- 52-week low: {q['52wk_low']:.2f} USD")

    if not lines:
        return "No reliable numeric data was retrieved."

    return "\n".join(lines)

In [5]:
# Data Agent Fetches company context, segments, market news, and competitive landscape directly from the LLM (no inputs needed except ticker).

class DataAgent:
    """Qualitative business context."""
    def run(self, ticker):
        prompt = f"""
Provide a well-structured qualitative overview of {ticker}, including:

1. Company overview
2. Major business segments
3. Industry context and secular trends
4. Competitive landscape
5. Key growth drivers
6. Recent business developments (high-level, factual)

Write clearly and concisely.
        """
        return call_llm(prompt)

In [6]:
# Financial Data Agent: This agent uses yfinance to gather numbers—not the user.

class FinancialDataAgent:
    """Real financial metrics via yfinance."""
    def run(self, ticker):
        stock = yf.Ticker(ticker)
        info = stock.info

        data = {
            "current_price": info.get("currentPrice"),
            "market_cap": info.get("marketCap"),
            "pe_ratio": info.get("trailingPE"),
            "eps_trailing": info.get("trailingEps"),
            "revenue_ttm": info.get("totalRevenue"),
            "gross_margins": info.get("grossMargins"),
            "operating_margins": info.get("operatingMargins"),
            "free_cash_flow": info.get("freeCashflow"),
            "52wk_high": info.get("fiftyTwoWeekHigh"),
            "52wk_low": info.get("fiftyTwoWeekLow"),
        }

        return data

In [7]:
# MarketTrendsAgent (Analyst Commentary + Trends)

class MarketTrendsAgent:
    """Analyst commentary & market trends."""
    def run(self, ticker, focus):
        prompt = f"""
Based on your trained knowledge, summarize current analyst commentary
and market trends impacting {ticker}. Provide:

1. Consensus analyst sentiment (bull vs bear themes)
2. Earnings call themes
3. Market/sector trends (AI spending, GPU demand, hyperscaler capex, energy intensity, etc.)
4. Competitive positioning commentary
5. Forward-looking expectations

FOCUS AREA REQUESTED BY USER (may be empty): {focus}

If a focus area is provided, emphasize trends and commentary most relevant to that theme
(e.g., 'AI & data center', 'gaming', 'automotive', 'Energy').

Make it structured, realistic, and insightful.
        """
        return call_llm(prompt)

In [8]:
# Analyst Agent: Transforms the DataAgent’s output into professional, investment-relevant analysis. (Combines Qualitative + Quantitative + Commentary)

class AnalystAgent:
    """Deep synthesis of qualitative + quantitative + commentary."""
    def run(self, qualitative, quantitative, commentary, focus):
        prompt = f"""
You are an Equity Research Analyst.

Combine the following information into a single in-depth ANALYST SUMMARY:

QUALITATIVE BUSINESS CONTEXT:
{qualitative}

QUANTITATIVE FINANCIAL DATA (with actual numbers):
{quantitative}

ANALYST COMMENTARY & MARKET TRENDS:
{commentary}

FOCUS AREA REQUESTED BY USER (may be empty): {focus}

Write a structured analysis with:

1. Business & segment interpretation
2. Financial health — explicitly cite key metrics (price, market cap, P/E, revenue, margins, FCF, etc.)
3. Growth drivers tied to real metrics where possible
4. Competitive strengths/weaknesses
5. AI infrastructure & secular tailwinds
6. Analyst sentiment interpretation
7. Where possible, distinguish between segment-level drivers
   (e.g., data center vs gaming vs other segments) in narrative form,
   noting which segments appear to be the main contributors to growth and profitability.

You MUST reference numeric values from the quantitative section directly in your writing
(e.g., “trading at ~54x P/E”, “TTM revenue of ~$165bn”, “gross margin near 70%”) whenever available.
        """
        return call_llm(prompt)

In [9]:
# Valuation Agent: Discusses multiples, growth expectations, margin profile, and qualitative valuation stance.

class ValuationAgent:
    """Valuation analysis."""
    def run(self, analyst_summary, quantitative, focus):
        prompt = f"""
You are a Valuation Specialist.

Using this analyst summary:
{analyst_summary}

And these financial metrics (with numbers):
{quantitative}

FOCUS AREA REQUESTED BY USER (may be empty): {focus}

Write a valuation discussion covering:

1. Relative valuation vs peers, explicitly citing P/E or other valuation metrics where available.
2. Profitability & margin profile — refer to gross/operating margins with actual percentages where available.
3. Revenue/EPS growth interpretation using the numbers.
4. How the requested focus area (if any) influences the valuation narrative
   (e.g., AI & data center growth vs gaming cyclicality vs energy-related demand).
5. Competitive/market-based valuation considerations.
6. Final valuation stance (no price target).

You MUST anchor your commentary in the numeric metrics above wherever possible.
        """
        return call_llm(prompt)

In [10]:
# Risk Reviewer Agent

class RiskReviewerAgent:
    """Independent risk review."""
    def run(self, valuation_discussion):
        prompt = f"""
You are a Risk Reviewer.

Based on this valuation discussion:
{valuation_discussion}

Identify key risks:

1. Regulatory & export controls
2. Supply chain dependencies
3. Competitive threats (AMD, in-house silicon, etc.)
4. Margin pressure & pricing
5. Macro risks and demand cyclicality

Be objective & specific.
        """
        return call_llm(prompt)

In [11]:
# Writer Agent (Sell-Side Research Report Generator) This generates the final report.

class WriterAgent:
    """Final sell-side-style research report."""
    def run(self, qualitative, quantitative, commentary, analyst, valuation, risks, ticker, focus):
        prompt = f"""
Write a full SELL-SIDE EQUITY RESEARCH REPORT on {ticker}.

Structure:

1. Executive Summary
2. Investment Thesis
3. Key Catalysts
4. Financial & Operating Snapshot
   - This section MUST present a bullet list of key metrics using the QUANTITATIVE DATA
     (share price, market cap, P/E, revenue, margins, 52-week range, etc.).
5. Valuation Discussion
6. Key Risks
7. Conclusion (Buy/Hold/Sell; NO price target)

QUALITATIVE INFO:
{qualitative}

QUANTITATIVE DATA (already formatted as bullet points with numbers):
{quantitative}

ANALYST COMMENTARY & TRENDS:
{commentary}

ANALYST SUMMARY:
{analyst}

VALUATION:
{valuation}

RISKS:
{risks}

FOCUS AREA REQUESTED BY USER (may be empty): {focus}

If a focus area is provided, weave this emphasis naturally throughout the thesis, catalysts,
and valuation narrative (e.g., highlight AI & data center economics, or gaming trends, or energy-related angles).

Produce a polished, realistic analyst report, and in the Financial & Operating Snapshot section,
you MUST include the numeric values explicitly as bullet points.
        """
        return call_llm(prompt, temperature=0.25)

In [12]:
# Reflection Agent (Improves the Final Report)

class ReflectionAgent:
    """Polish language & structure."""
    def run(self, report):
        prompt = f"""
Improve the clarity and professionalism of this equity research report.
Do NOT add price targets or change structure.

Report:
{report}
        """
        return call_llm(prompt, temperature=0.25)

In [13]:
# Supervisor Agent (Runs Entire Workflow)

class SupervisorAgent:

    def run(self, ticker="NVDA", focus=""):
        print("Gathering qualitative company data...")
        qualitative = DataAgent().run(ticker)

        print("Fetching real financial metrics...")
        quantitative_raw = FinancialDataAgent().run(ticker)
        print("Raw financial metrics:", quantitative_raw)
        quantitative = format_quantitative_for_prompt(quantitative_raw)

        print("Summarizing analyst commentary & market trends...")
        commentary = MarketTrendsAgent().run(ticker, focus)

        print("Creating deep analyst summary...")
        analyst = AnalystAgent().run(qualitative, quantitative, commentary, focus)

        print("Conducting valuation analysis...")
        valuation = ValuationAgent().run(analyst, quantitative, focus)

        print("Reviewing risks...")
        risks = RiskReviewerAgent().run(valuation)

        print("Writing the full equity research report...")
        report = WriterAgent().run(
            qualitative, quantitative, commentary, analyst, valuation, risks, ticker, focus
        )

        print("Polishing the final report...")
        improved_report = ReflectionAgent().run(report)

        return improved_report

In [15]:

# INTERACTIVE INPUT + RUN FULL SYSTEM

# Ask user for ticker
ticker = input("Enter the stock ticker you want an equity research report on (e.g., NVDA): ").strip().upper()

# Ask user for optional focus area
focus = input(
    "Optional: Enter a focus area for the analysis (e.g., 'AI & data center', 'gaming', 'automotive'), "
    "or leave blank for a general report: "
).strip()

supervisor = SupervisorAgent()

final_report = supervisor.run(ticker=ticker, focus=focus)

print("\n==============================")
print("FINAL EQUITY RESEARCH REPORT")
print("==============================\n")
print(final_report)

Enter the stock ticker you want an equity research report on (e.g., NVDA): NVDA
Optional: Enter a focus area for the analysis (e.g., 'AI & data center', 'gaming', 'automotive'), or leave blank for a general report: semiconductor and information technology
Gathering qualitative company data...
Fetching real financial metrics...
Raw financial metrics: {'current_price': 190.17, 'market_cap': 4630069116928, 'pe_ratio': 54.179485, 'eps_trailing': 3.51, 'revenue_ttm': 165217992704, 'gross_margins': 0.69847, 'operating_margins': 0.60842997, 'free_cash_flow': 52436750336, '52wk_high': 212.19, '52wk_low': 86.62}
Summarizing analyst commentary & market trends...
Creating deep analyst summary...
Conducting valuation analysis...
Reviewing risks...
Writing the full equity research report...
Polishing the final report...

FINAL EQUITY RESEARCH REPORT

# SELL-SIDE EQUITY RESEARCH REPORT: NVIDIA Corporation (NVDA)

## 1. Executive Summary
NVIDIA Corporation (NVDA) is a prominent leader in the semicond